# Presentation du cas

## Importations des librairies

In [1]:
# Standard libraries
import pandas as pd
import numpy as np
import sys
import os
import glob
import json
import warnings
import pickle
import string

# Graphic libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning libraries
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings('ignore')

## Telechargements des donnees

In [2]:
folder_path = os.path.join("..", "..", "data", "segmentation")
csv_files = glob.glob(folder_path + "/*.csv")
file_contents = []
for a in csv_files:
    file_contents.append((a, pd.read_csv(a)))

In [3]:
csv_files

['../../data/segmentation/olist_order_reviews_dataset.csv',
 '../../data/segmentation/olist_order_payments_dataset.csv',
 '../../data/segmentation/product_category_name_translation.csv',
 '../../data/segmentation/olist_products_dataset.csv',
 '../../data/segmentation/olist_orders_dataset.csv',
 '../../data/segmentation/olist_sellers_dataset.csv',
 '../../data/segmentation/olist_customers_dataset.csv',
 '../../data/segmentation/olist_order_items_dataset.csv',
 '../../data/segmentation/olist_geolocation_dataset.csv']

## Exploration initiale des donnees

In [4]:
print(file_contents[0][0])
file_contents[0][1].info()

../../data/segmentation/olist_order_reviews_dataset.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   review_id                100000 non-null  object
 1   order_id                 100000 non-null  object
 2   review_score             100000 non-null  int64 
 3   review_comment_title     11715 non-null   object
 4   review_comment_message   41753 non-null   object
 5   review_creation_date     100000 non-null  object
 6   review_answer_timestamp  100000 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [5]:
file_contents[0][1]["review_comment_message"]

0                                                      NaN
1                                                      NaN
2                                                      NaN
3                    Recebi bem antes do prazo estipulado.
4        Parabéns lojas lannister adorei comprar pela I...
                               ...                        
99995                                                  NaN
99996    Excelente mochila, entrega super rápida. Super...
99997                                                  NaN
99998    Solicitei a compra de uma capa de retrovisor c...
99999    meu produto chegou e ja tenho que devolver, po...
Name: review_comment_message, Length: 100000, dtype: object

Fichier contenant les avis de commande, contenant 100 000 observations dont pres de 40 000 ont un message et 10 000 ont un titre.

In [6]:
print(file_contents[1][0])
file_contents[1][1].info()

../../data/segmentation/olist_order_payments_dataset.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


Fichier contenant les commandes. Comme on pourrait si attendre, aucune donnee n'est absente de cette base.

In [7]:
print(file_contents[2][0])
file_contents[2][1].info()

../../data/segmentation/product_category_name_translation.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   product_category_name          71 non-null     object
 1   product_category_name_english  71 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


Traduction des produits en anglais.

In [8]:
print(file_contents[3][0])
file_contents[3][1].info()

../../data/segmentation/olist_products_dataset.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


Fichier contenant les descriptions de donnees. Les donnees sont presque entierement presentes mais deux observations n'ont qu'un numero d'identification.

In [9]:
print(file_contents[4][0])
file_contents[4][1].info()

../../data/segmentation/olist_orders_dataset.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


Fichier de suivi des commandes et de leur livraison. Les donnees sont majoritairement presentes, ce qui semble peu representatif des process de livraison reels.

In [10]:
print(file_contents[5][0])
file_contents[5][1].info()

../../data/segmentation/olist_sellers_dataset.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB


Fichier contenant les vendeurs de la plateforme. Les donnees sont entierement presentes.

In [11]:
print(file_contents[6][0])
file_contents[6][1].info()

../../data/segmentation/olist_customers_dataset.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


Fichier contenant les clients de la plateforme. Les donnees sont entierement presentes.

In [12]:
print(file_contents[7][0])
file_contents[7][1].info()

../../data/segmentation/olist_order_items_dataset.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


Fichier contenant les details des commandes. Les donnees sont entirement disponibles.

In [13]:
print(file_contents[8][0])
file_contents[8][1].info()

../../data/segmentation/olist_geolocation_dataset.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  int64  
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             1000163 non-null  object 
 4   geolocation_state            1000163 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 38.2+ MB


Fichier contenant des donnees de geolocalisation pour le bresil.

## New tables

Creation de nouvelles tables pour preparer les analyses suivantes

### Creation d'une table representant l'historique des commandes d'un client et des details pertinents

In [14]:
order_full = pd.merge(
    file_contents[4][1], file_contents[6][1], how="left", on="customer_id"
)
order_full = pd.merge(order_full, file_contents[7][1], how="left", on="order_id")
order_full = pd.merge(order_full, file_contents[1][1], how="left", on="order_id")


In [15]:
order_full = order_full.drop_duplicates(
    subset=["order_id", "customer_id", "product_id"]
)
order_full.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 103200 entries, 0 to 118433
Data columns (total 22 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       103200 non-null  object 
 1   customer_id                    103200 non-null  object 
 2   order_status                   103200 non-null  object 
 3   order_purchase_timestamp       103200 non-null  object 
 4   order_approved_at              103040 non-null  object 
 5   order_delivered_carrier_date   101398 non-null  object 
 6   order_delivered_customer_date  100195 non-null  object 
 7   order_estimated_delivery_date  103200 non-null  object 
 8   customer_unique_id             103200 non-null  object 
 9   customer_zip_code_prefix       103200 non-null  int64  
 10  customer_city                  103200 non-null  object 
 11  customer_state                 103200 non-null  object 
 12  order_item_id                 

### Product details full

In [16]:
order_product = pd.merge(
    file_contents[7][1], file_contents[3][1], how="left", on="product_id"
)
order_product = pd.merge(order_product, file_contents[0][1], how="left", on="order_id")


In [17]:
order_product = order_product.drop_duplicates(subset=["order_id", "product_id"])

order_product.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102425 entries, 0 to 113321
Data columns (total 21 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   order_id                    102425 non-null  object 
 1   order_item_id               102425 non-null  int64  
 2   product_id                  102425 non-null  object 
 3   seller_id                   102425 non-null  object 
 4   shipping_limit_date         102425 non-null  object 
 5   price                       102425 non-null  float64
 6   freight_value               102425 non-null  float64
 7   product_category_name       100965 non-null  object 
 8   product_name_lenght         100965 non-null  float64
 9   product_description_lenght  100965 non-null  float64
 10  product_photos_qty          100965 non-null  float64
 11  product_weight_g            102409 non-null  float64
 12  product_length_cm           102409 non-null  float64
 13  product_height

### Combination de products et orders

In [18]:
full = pd.merge(
    order_full,
    order_product.drop(
        [
            "order_item_id",
            "product_id",
            "seller_id",
            "shipping_limit_date",
            "price",
            "freight_value",
        ],
        axis=1,
    ),
    how="left",
    on="order_id",
)


In [19]:
full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112162 entries, 0 to 112161
Data columns (total 36 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       112162 non-null  object 
 1   customer_id                    112162 non-null  object 
 2   order_status                   112162 non-null  object 
 3   order_purchase_timestamp       112162 non-null  object 
 4   order_approved_at              112002 non-null  object 
 5   order_delivered_carrier_date   110316 non-null  object 
 6   order_delivered_customer_date  109071 non-null  object 
 7   order_estimated_delivery_date  112162 non-null  object 
 8   customer_unique_id             112162 non-null  object 
 9   customer_zip_code_prefix       112162 non-null  int64  
 10  customer_city                  112162 non-null  object 
 11  customer_state                 112162 non-null  object 
 12  order_item_id                 

In [20]:
full = full.drop_duplicates(subset=["order_id", "customer_id", "product_id"])
full.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 103200 entries, 0 to 112161
Data columns (total 36 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       103200 non-null  object 
 1   customer_id                    103200 non-null  object 
 2   order_status                   103200 non-null  object 
 3   order_purchase_timestamp       103200 non-null  object 
 4   order_approved_at              103040 non-null  object 
 5   order_delivered_carrier_date   101398 non-null  object 
 6   order_delivered_customer_date  100195 non-null  object 
 7   order_estimated_delivery_date  103200 non-null  object 
 8   customer_unique_id             103200 non-null  object 
 9   customer_zip_code_prefix       103200 non-null  int64  
 10  customer_city                  103200 non-null  object 
 11  customer_state                 103200 non-null  object 
 12  order_item_id                 

In [21]:
with open("order_full.pkl", "wb") as f:
    pickle.dump(
        order_full,
        f,
    )
with open("full.pkl", "wb") as f:
    pickle.dump(
        full ,
        f,
    )
    
with open("order_product.pkl", "wb") as f:
    pickle.dump(
        order_product ,
        f,
    )